This notebook reproduces the compilation results for employing SimuQ compilation on the quantum systems in the small benchmark on IBM devices. We employ IBM Quantum Experience here, so you need an IBM Quantum Experience account. For more details please refer to `README.md`. The compilations are executed on fake `ibmq_guadalupe` and `ibmq_washington` devices, which does not require specific access permissions. 

In [1]:
import time
import warnings 
warnings.filterwarnings('ignore', category=DeprecationWarning)

from simuq.ibm import IBMProvider
ibm=IBMProvider(from_file="../../../qiskit_APIKEY", hub="ibm-q-ornl", group="ornl", project="phy147")
#ibm = IBMProvider(api_key="YOUR_API_TOKEN", hub="ibm-q", group="open", project="main")

In [2]:
Trot = 4
Repitition = 1
tol = 0.5
T = 1

In [5]:
from qiskit import schedule
from qiskit import transpile as qiskit_transpile
from qiskit.providers.fake_provider import FakeGuadalupe, FakeWashington
from qiskit import QuantumCircuit
from qiskit.opflow import PauliTrotterEvolution
def run_benchmark(qs):
    n_qubits = len(qs.sites)
    sum_time = 0
    if n_qubits <= 27 :
        backend = "ibmq_guadalupe"
        fakebackend = FakeGuadalupe()
    else :
        backend = "ibmq_washington"
        fakebackend = FakeWashington()
    try:
        for _ in range(Repitition) :
            start_time = time.time()
            ibm.compile(qs, backend=backend, trotter_num=Trot, tol=tol, use_fake_backend=True, with_measure=False, verbose=-1)
            end_time = time.time()
            this_time = end_time - start_time
            sum_time += this_time
        print(f"Avgerage compilation time for {n_qubits} qubit using SimuQ", sum_time / Repitition)
        print(f"Pulse duration for {n_qubits} qubit using SimuQ", schedule(ibm.prog, ibm.backend).duration*0.22222*1e-3)
    except:
        print("Fail!")

    circ = QuantumCircuit(n_qubits)
    for evo_index in range(len(qs.evos)) :
        hamiltonian = qs.evos[evo_index][0].to_qiskit_opflow()
        evolution_op = (T*hamiltonian).exp_i()
        trotterized_op = PauliTrotterEvolution(
                        trotter_mode='trotter',
                        reps=Trot).convert(evolution_op)
        trot_op_circ = trotterized_op.to_circuit()
        circ = circ.compose(trot_op_circ)

    qiskit_sch = schedule(qiskit_transpile(circ, fakebackend), fakebackend)
    print(f"Pulse duration for {n_qubits} qubit using Qiskit", qiskit_sch.duration * 0.22222 * 1e-3)

In [ ]:
from simuq.systems.benchmark.ising_chain import GenQS
for N in [6,32,64,96]:
    qs = GenQS(N, T, 1, 1)
    run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.ising_cycle import GenQS
for N in [6,12,32,64]:
    qs = GenQS(N, T, 1, 1)
    run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.heis_chain import GenQS
N=32
qs = GenQS(N, T, 1, 1)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.qaoa_cycle import GenQS
N=12
qs = GenQS(N, 3,[1,1,1,1,1,1])
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.qhd import GenQS
N=16
qs = GenQS(N, T, 1, 1)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.mis_chain import GenQS
for N in [12,24]:
    qs = GenQS(N)
    run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.mis_grid import GenQS
for k in [4,5]:
    qs = GenQS(k)
    run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.kitaev import GenQS
N=18
qs = GenQS(N)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.schwinger import GenQS
N=10
qs = GenQS(N)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.o3nlσm import GenQS
N=5
M=6
qs = GenQS(N,M)
run_benchmark(qs)